In [21]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import collections
import os
import random
import numpy as np
from tqdm import tqdm
import sys, email
import pandas as pd
import math


In [22]:
ENRON_EMAIL_DATASET_PATH = 'enron.csv'

# load enron dataset
import pandas as pd

emails_df = pd.read_csv(ENRON_EMAIL_DATASET_PATH)
print(emails_df.shape)
emails_df.head()


(517401, 8)


,Message-ID,Date,From,To,Subject,Message,Cc,Bcc
0,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,NaN,Here is our forecast\n\n \n,NaN,NaN
1,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,Traveling to have a business meeting takes the...,NaN,NaN
2,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,test successful. way to go!!!\n,NaN,NaN
3,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,NaN,"Randy,\n\n Can you send me a schedule of the s...",NaN,NaN
4,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,Let's shoot for Tuesday at 11:45. \n,NaN,NaN


In [23]:
emails_sample_df = emails_df.copy()

In [24]:
# clean up subject line
emails_sample_df['Subject'] = emails_sample_df['Subject'].str.lower()
emails_sample_df['Subject'] = emails_sample_df['Subject'].str.replace('re:', ' ')
emails_sample_df['Subject'] = emails_sample_df['Subject'].str.replace('fw:', ' ')
#emails_sample_df['Subject'] = emails_sample_df['Subject'].str.replace(r'[^a-z]', ' ')
emails_sample_df['Subject'] = emails_sample_df['Subject'].str.replace(r'\s+', ' ')

# clean up content line
emails_sample_df['Message'] = emails_sample_df['Message'].str.lower()
emails_sample_df['Message'] = emails_sample_df['Message'].str.replace(r'[^a-z]', ' ')
emails_sample_df['Message'] = emails_sample_df['Message'].str.replace('fw:', ' ')
emails_sample_df['Message'] = emails_sample_df['Message'].str.replace(r'\s+', ' ')



C:\Users\Aaron\AppData\Local\Temp/ipykernel_12308/321775432.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  emails_sample_df['Subject'] = emails_sample_df['Subject'].str.replace(r'\s+', ' ')
C:\Users\Aaron\AppData\Local\Temp/ipykernel_12308/321775432.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  emails_sample_df['Message'] = emails_sample_df['Message'].str.replace(r'[^a-z]', ' ')
C:\Users\Aaron\AppData\Local\Temp/ipykernel_12308/321775432.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  emails_sample_df['Message'] = emails_sample_df['Message'].str.replace(r'\s+', ' ')


In [25]:
emails_sample_df.head()

,Message-ID,Date,From,To,Subject,Message,Cc,Bcc
0,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,NaN,here is our forecast,NaN,NaN
1,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,,traveling to have a business meeting takes the...,NaN,NaN
2,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,test,test successful way to go,NaN,NaN
3,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,NaN,randy can you send me a schedule of the salary...,NaN,NaN
4,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,hello,let s shoot for tuesday at,NaN,NaN


In [ ]:
emails_sample_df.head()

In [26]:
# create sentence list
emails_text = (emails_sample_df["Subject"] + ". " + emails_sample_df["Message"]).tolist()

In [27]:
emails_text

[nan,
 ' . traveling to have a business meeting takes the fun out of the trip especially if you have to prepare a presentation i would suggest holding the business plan meetings here then take a trip without any formal business meetings i would even try and get some honest opinions on whether a trip is even desired or necessary as far as the business meetings i think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not too often the presenter speaks and the others are quiet just waiting for their turn the meetings might be better if held in a round table discussion format my suggestion for where to go is austin play golf and rent a ski boat and jet ski s flying somewhere takes too much time ',
 ' test. test successful way to go ',
 nan,
 ' hello. let s shoot for tuesday at ',
 ' hello. greg how about either next tuesday or thursday phillip ',
 nan,
 ' prc review - phone calls. any morning between and ',
 ' high s

In [28]:
sentences = ' '.join([str(i) for i in emails_text])
words = sentences.split()

print('Data size', len(words))


Data size 141005095


In [29]:
# get unique words and map to glove set
print('Unique word count', len(set(words))) 

Unique word count 588751


In [30]:
# drop rare words
vocabulary_size = 10000

In [31]:
def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in tqdm(words):
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)

100%|███████████████████████████████████████████████████████████████| 141005095/141005095 [01:02<00:00, 2253406.53it/s]


In [32]:
del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 10063222], ('the', 5605630), ('to', 4006328), ('and', 2555652), ('of', 2358090)]
Sample data [965, 1260, 4047, 2, 25, 5, 104, 135, 1647, 1] ['nan', '.', 'traveling', 'to', 'have', 'a', 'business', 'meeting', 'takes', 'the']


In [33]:
data_index = 0
 
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [skip_window]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

In [34]:
batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

1260 . -> 4047 traveling
1260 . -> 965 nan
4047 traveling -> 2 to
4047 traveling -> 1260 .
2 to -> 25 have
2 to -> 4047 traveling
25 have -> 5 a
25 have -> 2 to


In [35]:
# Step 4: Build and train a skip-gram model.

batch_size = 128
embedding_size = 300  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.


In [36]:
graph = tf.Graph()


In [37]:
with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()


In [38]:
# Step 5: Begin training.
num_steps = 200001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print("Initialized")

  average_loss = 0
  for step in range(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step ", step, ": ", average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = "Nearest to %s:" % valid_word
        for k in range(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = "%s %s," % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  227.12051391601562
Nearest to or: rapid, sta, uniform, connection, odd, mtg., mailto, thailand,
Nearest to corp: alleging, open, suggested, trademark, publisher, audits, silver, communities,
Nearest to if: explosion, created, hmot, nyc, cure, predict, heidi, northwestern,
Nearest to energy: accordingly, benefit, management, convinced, washington, weekly, wake, celeste,
Nearest to s: cig, mgltd, nonetheless, finances, manipulation, republic, lozano, offline,
Nearest to us: arms, cisco, pressure, expects, nora, argentina, desks, integration,
Nearest to new: gillette, dana, imbalance, see, img, omnibus, dig, fielder,
Nearest to any: hong, adr, changes, isabel, dhou, stephen, contract., suspect,
Nearest to company: connects, loss, sachs, mean, especially, aol, anchor, epsc,
Nearest to of: links, negotiations, buck, red, opic, con, hereto, generators,
Nearest to its: multi, rybarski, riley, mastercard, estate, colleague, roger, program.,
Nearest to but

Average loss at step  52000 :  2.1718990309536457
Average loss at step  54000 :  2.7272606340274215
Average loss at step  56000 :  3.6755734808444975
Average loss at step  58000 :  3.7385874927043914
Average loss at step  60000 :  3.7311873428821563
Nearest to or: std, and, you, connection, uniform, msmith, afternoon, curves,
Nearest to corp: meta, com, development, alleging, north, std, open, enronxgate,
Nearest to if: when, with, resistance, nyc, std, although, for, claudia,
Nearest to energy: enron, ew, management, power, resistance, crawford, emissions, accordingly,
Nearest to s: is, std, resistance, and, has, gallon, was, relate,
Nearest to us: cisco, me, arms, desks, you, midnight, std, argentina,
Nearest to new: imbalance, gillette, see, std, fielder, ex, img, dana,
Nearest to any: the, its, your, std, changes, you, mostly, adr,
Nearest to company: connects, loss, notify, aol, epsc, sachs, mini, mistakes,
Nearest to of: for, std, in, font, with, oz, retire, bige,
Nearest to its:

Average loss at step  112000 :  3.700648647069931
Average loss at step  114000 :  3.677452382862568
Average loss at step  116000 :  3.6399139098227025
Average loss at step  118000 :  3.718252874433994
Average loss at step  120000 :  4.119159806370735
Nearest to or: and, std, connection, the, to, td, enron, pwcglobal,
Nearest to corp: enronxgate, com, meta, alleging, na, north, fid, americas,
Nearest to if: when, resistance, std, for, once, as, with, that,
Nearest to energy: power, enron, resistance, emissions, lawsuits, benefit, management, perrino,
Nearest to s: and, is, has, the, std, said, faxes, resistance,
Nearest to us: me, you, them, enron, the, midnight, argentina, her,
Nearest to new: imbalance, std, see, dana, ex, sweepsclub, img, of,
Nearest to any: the, some, your, a, you, no, its, or,
Nearest to company: enron, it, connects, sachs, loss, mean, person, aol,
Nearest to of: std, for, in, td, on, the, from, and,
Nearest to its: the, our, their, his, enron, mastercard, any, suc

Average loss at step  182000 :  3.637377890974283
Average loss at step  184000 :  3.8815501208901404
Average loss at step  186000 :  3.826226995050907
Average loss at step  188000 :  3.6702298680096863
Average loss at step  190000 :  3.6999279003143313
Nearest to or: std, enron, and, uniform, UNK, any, edith, images,
Nearest to corp: enronxgate, www, td, na, fid, com, hou, americas,
Nearest to if: when, resistance, thank, std, as, once, should, is,
Nearest to energy: power, wake, gas, celeste, hotmail, crawford, tree, violated,
Nearest to s: of, communications, the, and, development, has, com, td,
Nearest to us: me, them, him, over, you, her, passenger, midnight,
Nearest to new: see, imbalance, gillette, std, and, mcnamara, global, fourth,
Nearest to any: some, the, no, your, a, changes, or, its,
Nearest to company: connects, person, it, companies, aspect, coordinated, mean, seriously,
Nearest to of: for, in, std, and, on, s, td, to,
Nearest to its: their, our, the, his, your, any, suc

In [39]:
# create a dataframe using the embedded vectors and attach the key word as row header
import pandas as pd
enrond_dataframe = pd.DataFrame(final_embeddings)
enrond_dataframe = enrond_dataframe.set_index([list(reverse_dictionary.values())])


In [40]:
# See what it learns and look at clusters to pull out major themes in the data
CLUSTER_SIZE = 300 
# cluster vector and investigate top groups
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=CLUSTER_SIZE)
cluster_make = kmeans.fit_predict(enrond_dataframe)

labels = kmeans.predict(enrond_dataframe)
import collections
cluster_frequency = collections.Counter(labels)
print(cluster_frequency)
cluster_frequency.most_common()

clusters = {}
n = 0
for item in labels:
    if item in clusters:
        clusters[item].append(list(enrond_dataframe.index)[n])
    else:
        clusters[item] = [list(enrond_dataframe.index)[n]]
    n +=1

for k,v in cluster_frequency.most_common(100):
  print('\n\n')
  print('Cluster:', k)
  print (' '.join(clusters[k]))

Counter({13: 264, 10: 134, 81: 102, 297: 96, 238: 93, 16: 89, 187: 86, 103: 83, 293: 79, 126: 78, 42: 72, 282: 70, 130: 69, 97: 69, 47: 68, 29: 67, 54: 65, 254: 60, 168: 59, 274: 58, 112: 58, 18: 58, 278: 57, 108: 57, 147: 55, 253: 54, 73: 51, 110: 51, 275: 51, 87: 49, 221: 49, 146: 49, 49: 49, 167: 49, 11: 48, 178: 48, 177: 48, 48: 48, 200: 48, 99: 46, 280: 46, 134: 45, 206: 45, 120: 45, 263: 45, 15: 44, 46: 44, 219: 44, 113: 44, 75: 44, 40: 43, 271: 43, 196: 42, 258: 42, 232: 42, 233: 41, 84: 41, 180: 41, 141: 40, 35: 40, 249: 40, 137: 40, 139: 39, 79: 39, 215: 39, 286: 39, 172: 39, 246: 39, 179: 39, 247: 39, 175: 39, 123: 38, 77: 38, 239: 37, 93: 37, 20: 37, 158: 36, 195: 36, 169: 36, 41: 36, 57: 36, 68: 36, 298: 36, 80: 36, 114: 36, 71: 35, 85: 35, 44: 35, 74: 35, 102: 35, 105: 35, 164: 35, 202: 35, 32: 35, 203: 35, 131: 34, 23: 34, 267: 34, 118: 34, 26: 34, 3: 33, 101: 33, 148: 33, 156: 33, 290: 33, 117: 33, 33: 33, 245: 32, 30: 32, 19: 32, 244: 32, 208: 32, 149: 32, 17: 32, 4: 32